# COGS 108 - Final Project 

# Overview

The intersection of crime and community can often be puzzling, bringing into question which influences the other more and positively or negatively? In this project, with public parks as a representation of a community, I aimed to understand how crime in public parks in the San Diego area affects its attractiveness to locals and tourists or vice versa. To do so, I analyzed the relationship between Yelp ratings of San Diego public parks and the number of police dispatches in the area around said parks. From this analysis, the data suggests that

# Name & GitHub

-  Name: Giselle Mejia
- GitHub Username: 9GCMEJIA

# Research Question

Is there a relationship between the frequency of police call dispatches in areas around San Diego public parks and the Yelp ratings of such associated parks?

## Background and Prior Work

San Diego Parks and Recreation accounts for overseeing nearly all public space including 40,000 acres of developed and undeveloped open space, 340 parks, 25 miles of shoreline, 57 recreation centers, 13 pools, 7 skate parks, 3 golf courses. Their overall mission is: "To provide healthy, sustainable, and enriching environments for all." Their vision on the other hand is "To connect all to the city’s diverse world class park system." In past studies, scholars have explored the relationship between park facilities, observed park use, and physical activity levels, but did not stray too far away to analyze the intersecting trends and consequences between the parks and the neighboring community. 

Yelp is a popular ratings and review platform for many public and private facilities and provides a good insight into the general community opinion of the local San Diego parks. This includes ratings on 5 star scale and written reviews.

While there are not many studies looking into the comparison of crime in public parks and data from the community (Yelp, in this case), there have been speculations and observations made claiming that parks act as crime inhibitors or generators. This notion relies on the idea that "nearby land uses and socio-demographic characteristics are a key driver of crime being located within the park or nearby the park." San Diego, specifically, has recently began instilling park curfews in response to many crime concerns around the Pacific Beach/Mission Bay area. Spiked crime in parks may dwell in the very nature that parks are public, available to all, and often unsupervised. On the otherhand, parks are also said to be very "critical" for low income or underprivileged communities, providing a free space to congregate or spend leisure time. Thus, in the eyes of the community parks can be a blessing or curse.


References:
- https://www.sandiego.gov/parks-and-recreation
- https://www.sandiegouniontribune.com/communities/san-diego/story/2020-02-16/park-curfews-on-the-rise-in-san-diego-in-response-to-crime-concerns
- https://www.sciencedirect.com/science/article/abs/pii/S0049089X18301303 
- https://js.sagamorepub.com/jpra/article/view/6078

# Hypothesis


I hypothesize that a higher frequency of local crime near San Diego public parks correlate to a lower Yelp rating, while lower frequencies will correlate to a higher rating. I predict this case because Yelp reviews are often both locals and tourists--everyday citizens--rather than official inspectors and thus they overlap with the population of locals who also make the police calls. This may point to a more negative outlook by the community who has to deal with the rampant crime.

# Dataset(s)

- Dataset Name: Dataset (CSV, Park Information): yelp_SD_parks.csv
- Source: Professor Ellis
- Link to the dataset: https://www.yelp.com/developers/documentation/v3/business_search
- Number of observations: 834
This dataset includes general information about San Diego park names, addresses, phone numbers, and websites.

- Dataset Name: Dataset (CSV, Reviews): yelp_SD_reviews.csv
- Source: Professor Ellis
- Link to the dataset: https://www.yelp.com/developers/documentation/v3/business_reviews
- Number of observations: 2334
This dataset includes observed numeric ratings and written text reviews about San Diego parks through Yelp.

- Dataset Name: Dataset (CSV, 2019): pd_calls_for_service_2019_datasd.csv
- Source: Professor Ellis
- Link to the dataset: https://data.sandiego.gov/datasets/police-calls-for-service/
- Number of observations:
This dataset includes information about the calls dispatched by the SD Police Department, showing dates, addresses, call types, and priority levels. I am using data from only the year 2019 and not 2020 to get a better understanding without the possible factor of COVID-19 limitations and influences on the 2020 data.

To combine these datasets, I plan on merging all the information through the common name and address of the parks used in all the sets.


# Setup

In [3]:
# imports
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import patsy
import requests

In [ ]:
# read in csv to dataframes
parks_df = pd.read_csv()
reviews_df = pd.read_csv()
police_calls_df = pd.read_csv()

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

The information by Yelp and the police department are public record and all individual names and identifiers for people are not present, thus giving any individuals involved privacy.

However, while my question does not directly involve any evident ethical steps, the misuse of such data can generate ethical and privacy concerns. The ethical ambiguity revolves around some implicit biases brought about by the data. This information can point out hotspots for crime in the San Diego area and thus bring about some stigmas against a neighborhood or peoples from such neighborhood.

There is also bias in our datasets themselves for it does not completely capture the public view only those who are involved in the community and online through Yelp. In addition, the police dispatch dataset deliberately leaves out data regarding sensitive incidents including "domestic violence, child abuse, suicide, sex crimes and stalking." Thus, we cannot get the entire picture by this data alone, but perhaps very close to it.

# Conclusion & Discussion

*Fill in your discussion information here*